In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install edgeimpulse

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 28.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.9/78.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 10.7 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.5.0
    Uninstalling urllib3-2.5.0:
      Successfully uninstalled urllib3-2.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 4.

In [ ]:
!pip install edgeimpulse-api

In [4]:
import os
import numpy as np
import edgeimpulse as ei

# ------------------------------
# 🔧 CONFIGURATION
# ------------------------------
EI_API_KEY = "ei_eb03aa1fa9592e163b72614732eecc5756bc5250c1778d48150ca837df74ca23"  # Replace with your actual Edge Impulse API key
npz_folder = r"/kaggle/input/noiser/noise"  # Folder with all .npz files
SAMPLE_RATE_HZ = 100  # adjust to your actual sample rate
CATEGORY = "training"  # or "testing"

ei.API_KEY = EI_API_KEY  # authenticate Edge Impulse SDK

# ------------------------------
# ⚙️ Helper function to load waveforms
# ------------------------------
def load_npz_waveforms(folder_path):
    data_list = []
    labels = []
    filenames = []

    for file in os.listdir(folder_path):
        if not file.endswith(".npz"):
            continue

        file_path = os.path.join(folder_path, file)
        try:
            d = np.load(file_path, allow_pickle=True)
            waveform = d["waveform"]  # shape (N, 3)
            attributes = d["attributes"].item()

            label = attributes.get("trace_category", None)
            if label not in ["earthquake_local", "noise"]:
                print(f"⚠️ Skipping {file}: invalid or missing label")
                continue

            # ensure float32 for upload
            waveform = np.array(waveform, dtype=np.float32)

            data_list.append(waveform)
            labels.append(label)
            filenames.append(file)

        except Exception as e:
            print(f"⚠️ Failed to load {file_path}: {e}")

    return data_list, labels, filenames

# ------------------------------
# 📂 Load all NPZ data
# ------------------------------
print("📂 Loading data...")
waveforms, labels, files = load_npz_waveforms(npz_folder)
print(f"✅ Loaded {len(waveforms)} samples from {npz_folder}")

if len(waveforms) == 0:
    raise RuntimeError("No valid NPZ files found. Check folder path or file format.")

# ------------------------------
# 📤 Upload to Edge Impulse
# ------------------------------
sample_rate_ms = 1000.0 / SAMPLE_RATE_HZ

sensors = [
    {"name": "E_channel", "units": "counts"},
    {"name": "N_channel", "units": "counts"},
    {"name": "Z_channel", "units": "counts"}
]

print("🚀 Uploading to Edge Impulse...")
resp = ei.experimental.data.upload_numpy(
    sample_rate_ms=sample_rate_ms,
    data=np.array(waveforms, dtype=object),  # variable-length allowed
    labels=labels,
    sensors=sensors,
    metadata={"source": "seismic_dataset"}
)

# ------------------------------
# 🧾 Upload Results
# ------------------------------
print("✅ Upload Summary:")
print("   Successful uploads:", len(resp.successes))
print("   Failed uploads:", len(resp.fails))

if resp.fails:
    for i, fail in enumerate(resp.fails):
        print(f"❌ {files[i]} → {fail}")
else:
    print("🎉 All NPZ samples uploaded successfully!")


2025-10-28 17:22:58.668112: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761672178.876106      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761672178.936246      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


📂 Loading data...
✅ Loaded 2236 samples from /kaggle/input/noiser/noise
🚀 Uploading to Edge Impulse...
✅ Upload Summary:
   Successful uploads: 2236
   Failed uploads: 0
🎉 All NPZ samples uploaded successfully!
